<a href="https://colab.research.google.com/github/MattJCR/Red-neuronal-Basica/blob/master/Red_neuronal_Basica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

In [ ]:
class NetNode(object):
  def __init__(self):
    self.inputs = []
    self.weights = []
    self.value = None

class Network(object):

  def __init__(self, layers):

    self.net = [[NetNode() for _ in range(size)] for size in layers]

    sizes = len(layers)

    for layer in range(1,sizes):
      for node in self.net[layer]:
        for unit in self.net[layer - 1]:
          node.inputs.append(unit)
          node.weights.append(0)
  
  def relu(self,z):
    return max(0,z)

  def relu_prime(self, z):
    return 1 if z > 0 else 0

  def backpropagation(self, eta, examples, epochs):
        inputs = self.net[0]
        outputs = self.net[-1]
        layer_size = len(self.net)
        for layer in self.net[1:]:
            for node in layer:
                node.weights = [np.random.uniform() for _ in range(len(node.weights))]
        for epoch in range(epochs):
            for x_train, y_train in examples:
                for value, node in zip(x_train, inputs):
                    node.value = value
                for layer in self.net[1:]:
                    for node in layer:
                        in_val = [n.value for n in node.inputs]
                        unit_value = np.dot(in_val, node.weights)
                        node.value = self.relu(unit_value)
                delta = [[] for _ in range(layer_size)]
                err = [y_train[i] - outputs[i].value for i in range(len(outputs))]
                delta[-1] = [self.relu_prime(outputs[i].value) * err[i] for i in range(len(outputs))]
                hidden_layers = layer_size - 2
                for i in range(hidden_layers, 0, -1):
                    layer = self.net[i]
                    n_layers = len(layer)
                    w = [[node.weights[l] for node in self.net[i + 1]] for l in range(n_layers)]
                    delta[i] = [self.relu_prime(layer[j].value) * np.dot(w[j], delta[i + 1]) for j in range(n_layers)]
                for i in range(1, layer_size):
                    layer = self.net[i]
                    in_val = [node.value for node in self.net[i - 1]]
                    n_layers = len(self.net[i])
                    for j in range(n_layers):
                        layer[j].weights = np.add(layer[j].weights, np.multiply(eta * delta[i][j], in_val))
            # print(f"epoch {epoch}/{epochs} | total error={np.sum(err)/len(examples)}")
    
  def predict(self,input_data):
    inputs = self.net[0]

    for v, n in zip(input_data, inputs):
      n.value = v

    for layer in self.net[1:]:
      for node in layer:
        in_val = [n.value for n in node.inputs]
        unit_value = np.dot(in_val, node.weights)
        node.value = self.relu(unit_value)
    
    outputs = self.net[-1]
    return outputs.index(max(outputs, key=lambda node: node.value))

  def accuracy(self, examples):
    correct = 0
    for x_test, y_test in examples:
      prediction = self.predict(x_test)

      if(y_test[prediction] == 1):
        correct += 1
    return correct / len(examples)

In [ ]:
iris_X,iris_y = datasets.load_iris(return_X_y = True)

In [ ]:
iris_X[:10]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1]])

In [ ]:
iris_y[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
iris_x_normalized = normalize(iris_X,axis=0)
iris_x_normalized[:10]

array([[0.07056264, 0.09254209, 0.02754801, 0.01150242],
       [0.06779548, 0.07932179, 0.02754801, 0.01150242],
       [0.06502832, 0.08460991, 0.02558029, 0.01150242],
       [0.06364474, 0.08196585, 0.02951572, 0.01150242],
       [0.06917906, 0.09518615, 0.02754801, 0.01150242],
       [0.07471338, 0.10311833, 0.03345115, 0.02300485],
       [0.06364474, 0.08989803, 0.02754801, 0.01725364],
       [0.06917906, 0.08989803, 0.02951572, 0.01150242],
       [0.06087757, 0.07667773, 0.02754801, 0.01150242],
       [0.06779548, 0.08196585, 0.02951572, 0.00575121]])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(iris_x_normalized,iris_y,test_size=0.2, shuffle=True)

In [ ]:
y_train = np_utils.to_categorical(y_train,num_classes=3)
y_test = np_utils.to_categorical(y_test,num_classes=3)

In [ ]:
examples = []
for i in range(len(X_train)):
  examples.append([X_train[i],y_train[i]])

In [ ]:
net = Network([4,7,3])

In [ ]:
net.backpropagation(0.03,examples,100)

In [ ]:
examples = []
for i in range(len(X_test)):
  examples.append([X_test[i],y_test[i]])

In [ ]:
prediction = net.predict(X_test[0])
accuracy = net.accuracy(examples)

print(f"Accuracy:  {accuracy}")
print(f"Desired output:  {y_test[0]}")
print(f"Index of output:  {prediction}")

Accuracy:  1.0
Desired output:  [0. 1. 0.]
Index of output:  1
